In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import s3fs
import numpy as np
import fireducks.pandas as pd
import warnings
from dotenv import load_dotenv
from tqdm import tqdm
from pprint import pprint
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import (
                                    cross_val_score,
                                    RandomizedSearchCV,
                                    GridSearchCV, 
                                    train_test_split,
                                    StratifiedKFold
                                    )
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from gensim.models import Word2Vec, FastText

sys.path.append("../src")
from ml_utils import *

/opt/conda/lib/python3.12/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/onyxia/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 40.3 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
load_dotenv()
pd.set_option("display.max_columns", None)
warnings.simplefilter("ignore")
fs = s3fs.S3FileSystem(
            client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"},
            key=os.environ["Accesskey"],
            secret=os.environ["Secretkey"],
            token=os.environ["Token"]
)

# Data Preprocessing

In [4]:
with fs.open("elissamim/text_classification_men/data/stages-votes.json", "r") as file:
    df = pd.read_json(file)

df = df.groupby("phrase_text", as_index = False)["sol"].apply(lambda x: x.mode().iloc[0])
df["sol"]=df["sol"].apply(lambda x: 1 if x == "ok" else 0)
df["clean_phrase_text"] = df["phrase_text"].apply(lambda x: nltk_text_preprocessing(x, True))
df = df[df["clean_phrase_text"] != ""]
df.head()

,phrase_text,sol,clean_phrase_text
0,* Aider à la mise en place de l évènement Shar...,0,aider mise place évènemer shareplan envoi rapp...
1,* Comprendre le métier des achats * Comment or...,0,comprendre métier achat comment organiser appe...
2,* Fendre du bois en forêt au merlin manuelleme...,0,fendre boi forêt merlin manuellemer débarder b...
4,"2 jours au CDI , 1 jour en arts plastiques , 1...",0,2 jour cdi 1 jour art plastique 1 jour musiqu ...
5,4 jours au sein du Bureau des affaires institu...,1,4 jour sein bureau affaire institutionnel fina...


# Model selection (static embedding (sparse or dense) + classification algorithm) with grid search and cross validation

In [5]:
X = df["clean_phrase_text"]
y = df["sol"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state = 42)

In [6]:
tokenized_texts = [text.split() for text in X_train]
word2vec_model = Word2Vec(sentences = tokenized_texts,
                         vector_size = 100,
                         window = 5,
                         min_count = 1,
                         workers = 4,
                         seed = 42)
fasttext_model = FastText(sentences = tokenized_texts,
                         vector_size = 100,
                         window = 5,
                         min_count = 1,
                         workers = 4,
                         seed = 42)

static_embedding_models = {
    # Sparse embeddings
    "Bag of Words":CountVectorizer(),
    "TF":TfidfVectorizer(use_idf=False),
    "TF-IDF":TfidfVectorizer(),
    # Dense embeddings
    "Word2Vec": MeanEmbeddingVectorizer(model=word2vec_model),
    "FastText": MeanEmbeddingVectorizer(model=fasttext_model)
}

classification_models = {
    "Logistic Regression":LogisticRegression(),
    "Random Forest":RandomForestClassifier(),
    "Linear SVM":SVC(kernel="linear", probability=True),
    "Multinomial Naive Bayes":MultinomialNB(),
    "XGBoost":XGBClassifier(use_label_encoder=False, eval_metric="logloss")
}

In [7]:
params_grid = {
    "Bag of Words" : {
        "feature_extraction__ngram_range":[(1,1), (1,2), (2,2)],
        "feature_extraction__max_df":[.7, .9, 1],
        "feature_extraction__min_df":[.001, .01, .1],
        "feature_extraction__max_features":[10, 100, 1000, 10000],
        "feature_extraction__binary":[True, False]
    },
    "TF" : {
        "feature_extraction__max_df":[.7, .8, .9, 1],
        "feature_extraction__min_df":[.001, .01, .1],
        "feature_extraction__norm":["l1", "l2", None],
        "feature_extraction__sublinear_tf":[True, False],
        "feature_extraction__max_features":[10, 100, 1000, 10000],
        "feature_extraction__ngram_range":[(1,1), (1,2), (2,2)]
    },
    "TF-IDF" : {
        "feature_extraction__max_df":[.7, .8, .9, 1],
        "feature_extraction__min_df":[.001, .01, .1],
        "feature_extraction__norm":["l1", "l2", None],
        "feature_extraction__sublinear_tf":[True, False],
        "feature_extraction__max_features":[10, 100, 1000, 10000],
        "feature_extraction__ngram_range":[(1,1), (1,2), (2,2)]
    },
    "Logistic Regression":{
        "classifier__C":[.001, .01, .1, 1, 10, 100],
        "classifier__penalty":["l2"],
        "classifier__solver":["lbfgs"]
    },
    "Random Forest":{
        "classifier__n_estimators":[100,300,500],
        "classifier__max_depth":[None, 10, 20, 50],
        "classifier__min_samples_split": [2, 5, 10],
        "classifier__min_samples_leaf":[1, 2, 5, 10],
        "classifier__max_features":["sqrt", "log2", None],
        "classifier__bootstrap":[True, False]
    },
    "Linear SVM":{
        "classifier__C":[.001, .01, .1, 1, 10, 100]
    },
    "Multinomial Naive Bayes":{
        "classifier__alpha":[.0001, .001, .01, .1, 1, 10],
        "classifier__fit_prior":[True, False]
    },
    "XGBoost":{
        "classifier__n_estimators":[100, 200, 300, 500],
        "classifier__max_depth":[3, 5, 7, 10],
        "classifier__learning_rate":[.01, .05, .1, .3],
        "classifier__subsample":[.5, .7, .8, 1],
        "classifier__colsample_bytree":[.5, .7, .8, 1],
        "classifier__gamma":[0, .1, .5, 1],
        "classifier__reg_alpha":[0, .01, .1, 1],
        "classifier__reg_lambda":[.1, 1, 10]
    }
}

In [9]:
dict_scores, dict_params = {}, {}

for embedding_name, embedding_model in tqdm(static_embedding_models.items(),
                                           desc="Static embeddings"):

    dict_scores[embedding_name], dict_params[embedding_name] = {}, {}
    
    for classification_name, classification_model in tqdm(classification_models.items(),
                                                         desc="Classification algorithms"):

        # Multinomial NB is not suited for dense vectors
        if embedding_name in ["Word2Vec", "FastText"] and classification_name == "Multinomial Naive Bayes":
            continue

        steps = [("feature_extraction", embedding_model)]

        # For Logistic Regression and Linear SVM, and for dense embeddings, add standardisation
        if embedding_name in ["Word2Vec", "FastText"] and classification_name in ["Logistic Regression", "Linear SVM"]:
            steps.append(("standardisation", StandardScaler()))

        steps.append(("classifier", classification_model))

        pipeline = Pipeline(steps)

        # We compute scores using Grid Search on the parameter grid to do model selection with best
        # hyperparameters
        cv = StratifiedKFold(n_splits=5, 
                                 shuffle = True, 
                                 random_state=42)
        
        if (embedding_name in params_grid) or (classification_name in params_grid):

            selected_params_grid = {
                **params_grid.get(embedding_name, {}),
                **params_grid.get(classification_name, {})
            }

            try:
                grid_search = RandomizedSearchCV(
                    pipeline,
                    param_distributions=selected_params_grid,
                    cv=cv,
                    scoring="accuracy",
                    n_jobs=-1,
                    verbose=0,
                    n_iter = 20
                )
            except PicklingError:
                grid_search = RandomizedSearchCV(
                    pipeline,
                    param_distributions=selected_params_grid,
                    cv=cv,
                    scoring="accuracy",
                    n_jobs=1,
                    verbose=0,
                    n_iter = 20
                )
            
            grid_search.fit(X_train, y_train)
            
            score = grid_search.best_score_
            dict_params[embedding_name][classification_name] = grid_search.best_params_

        else:

            scores = cross_val_score(
                pipeline,
                X_train, 
                y_train,
                cv=cv,
                scoring="accuracy"
            )

            score = np.mean(scores)
            
        dict_scores[embedding_name][classification_name] = score

Classification algorithms:  80%|████████  | 4/5 [00:40<00:10, 10.04s/it]/opt/conda/lib/python3.12/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have insta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--  Downloading https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.8.0/fr_core_news_sm-3.8.0-py3-none-any.whl (16.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--  Downloading https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.8.0/fr_core_news_sm-3.8.0-py3-none-any.whl (16.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--  Downloading https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.8.0/fr_core_news_sm-3.8.0-py3-none-any.whl (16.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--  Downloading https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.8.0/fr_core_news_sm-3.8.0-py3-none-any.whl (16.

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/onyxia/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/onyxia/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 11.0 MB/s eta 0:00:000:00:02m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━ 13.1/16.3 MB 8.1 MB/s eta 0:00:01

/opt/conda/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:33:56] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1744352472081/work/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 9.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 9.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 8.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 8.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 8.2 MB/s eta 0:00:00


/opt/conda/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:33:56] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1744352472081/work/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/conda/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:33:56] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1744352472081/work/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/conda/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:33:56] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1744352472081/work/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/conda/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:33:56] WARNING: /home/conda/feedstock_root/build

PicklingError: Could not pickle the task to send it to the workers.

In [12]:
classification_name

'Logistic Regression'

In [10]:
pprint(dict_scores)

{'Bag of Words': {'Linear SVM': 0.6669811320754716,
                  'Logistic Regression': 0.6839622641509434,
                  'Multinomial Naive Bayes': 0.679245283018868,
                  'Random Forest': 0.690566037735849,
                  'XGBoost': 0.689622641509434},
 'FastText': {},
 'TF': {'Linear SVM': 0.6867924528301887,
        'Logistic Regression': 0.6528301886792454,
        'Multinomial Naive Bayes': 0.6820754716981131,
        'Random Forest': 0.6783018867924528,
        'XGBoost': 0.6962264150943396},
 'TF-IDF': {'Linear SVM': 0.6839622641509433,
            'Logistic Regression': 0.6886792452830189,
            'Multinomial Naive Bayes': 0.630188679245283,
            'Random Forest': 0.6877358490566038,
            'XGBoost': 0.6547169811320754},
 'Word2Vec': {'Linear SVM': 0.6028301886792452,
              'Logistic Regression': 0.6132075471698114,
              'Random Forest': 0.6632075471698113,
              'XGBoost': 0.6632075471698113}}


# Model evaluation

In [ ]:
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("logreg", LogisticRegression(max_iter=1000))
])



grid_search = GridSearchCV(
    pipeline,
    params_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    verbose=0
)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)

In [ ]:
y_pred = grid_search.best_estimator_.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
confusion_matrix(y_test, y_pred)

In [ ]:
joblib.dump(grid_search.best_estimator_,
            "../models/tfidf_logreg_model.joblib")